In [8]:
!pip install yfinance datetime plotly matplotlib requests
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import yfinance as yf
import plotly
import datetime




DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.7/27.7 MB 13.8 MB/s eta 0:00:0000:0100:01
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near fu

In [9]:
start = datetime.datetime(2016,1,1)
end = datetime.datetime.today()


In [12]:
data = yf.download(tickers='BTC-USD', start=start, end=end, interval='1d')

[*********************100%***********************]  1 of 1 completed


In [13]:
data['Log Diff'] = np.log(data['Close']) - np.log(data['Open'])

In [20]:
data['Change'] =  ((data['Close'] - data['Open'])/data['Open']) * 100

In [21]:
data

,Open,High,Low,Close,Adj Close,Volume,log_diff,Change
Date,,,,,,,,
2015-12-31,425.875000,432.920990,418.734985,430.566986,430.566986,45996600,0.010957,1.101728
2016-01-01,430.721008,436.246002,427.515015,434.334015,434.334015,36278900,0.008353,0.838828
2016-01-02,434.622009,436.062012,431.869995,433.437988,433.437988,30096600,-0.002728,-0.272425
2016-01-03,433.578003,433.743011,424.705994,430.010986,430.010986,39633800,-0.008261,-0.822693
2016-01-04,430.061005,434.516998,429.084015,433.091003,433.091003,38477500,0.007021,0.704551
...,...,...,...,...,...,...,...,...
2022-03-21,41246.132812,41454.410156,40668.042969,41077.996094,41077.996094,24615543271,-0.004085,-0.407642
2022-03-22,41074.105469,43124.707031,40948.281250,42358.808594,42358.808594,32004652376,0.030799,3.127769
2022-03-23,42364.378906,42893.507812,41877.507812,42892.957031,42892.957031,25242943069,0.012400,1.247695
